# Opacity Databases 

``PICASO`` currently comes with single opacity database that includes both the continuum and molecular opacity. The opacity file is already included in the `reference/` directory. You may have to install [Github Large File System](https://git-lfs.github.com) if you notice that `opacity.db` is downloading as an empty file. 

We chose those use `sqlite3` for our database because of it's 1) user-friendliness, 2) speed, 3) scalability, 4) compatibility with parallel programing. We tried out various other methods as well-- `json`, `hdf5`, `ascii`, `sqlalchemy`-- but `sqlite3` was truly better for this specific problem.  

This tutorial shows you how query the existing `opacity.db` and also shows you how to customize your own `opacity.db`

In [1]:
import sqlite3
import io
import numpy as np
import os

## General Sqlite3 and how our database is structured

- 3 Tables: `header`, `continuum`, and `molecular`
- header: contains units and the wavenumber grid
- continuum: contains a grid continuum opacity and is temperature dependent 
- molecular: contains all molecular opacity and is pressure-temperature dependent. 

## How to Query the Database

In [2]:
#this is where your opacity file should be located if you've set your environments correctly
db_filename = os.path.join(os.getenv('picaso_refdata'), 'opacities','opacity.db')
#this is how you establish a connection to the db
conn = sqlite3.connect(db_filename, detect_types=sqlite3.PARSE_DECLTYPES)

#these functions are so that you can store your float arrays as bytes to minimize storage
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

#tell sqlite what to do with an array
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

In [3]:
#this will be how we execute commands to grab chunks of data
cur = conn.cursor()

### Get header info

In [4]:
#let's start by just grabbing all the info from the header
header = cur.execute('SELECT * FROM header')
cols = [description[0] for description in header.description]
data = cur.fetchall()
cols, data

(['id',
  'pressure_unit',
  'temperature_unit',
  'wavenumber_grid',
  'continuum_unit',
  'molecular_unit'],
 [(1,
   'bar',
   'kelvin',
   array([10003.5012254, 10007.0049034, 10010.5110366, ..., 33255.7366146,
          33294.4897619, 33333.3333333]),
   'cm-1 amagat-2',
   'cm2/molecule')])

### Get Continuum Opacity

In [5]:
#what molecules exist? 
cur.execute('SELECT molecule FROM continuum')
print(np.unique(cur.fetchall()))

['H-bf' 'H-ff' 'H2-' 'H2CH4' 'H2H' 'H2H2' 'H2He' 'H2N2']


In [6]:
#what temperatures exist?
cur.execute('SELECT temperature FROM continuum')
cia_temperatures = np.unique(cur.fetchall())
cia_temperatures[0:10]

array([ 75., 100., 125., 150., 175., 200., 225., 250., 275., 300.])

In [7]:
#wavenumber grid from header 
cur.execute('SELECT wavenumber_grid FROM header')
wave_grid = cur.fetchone()[0]

In [8]:
#grab H2H2 at 300 K 
cur.execute('SELECT opacity FROM continuum WHERE molecule=? AND temperature=?',('H2H2',300))
data = cur.fetchall()
data

[(array([3.23076970e-14, 2.32440540e-18, 1.66117181e-22, ...,
         7.33735036e-51, 6.12487537e-51, 5.11059697e-51]),)]

In [9]:
#grab all opacity at 300 K 
cur.execute('SELECT molecule,opacity FROM continuum WHERE temperature=300')
data = cur.fetchall()
data

[('H2H2', array([3.23076970e-14, 2.32440540e-18, 1.66117181e-22, ...,
         7.33735036e-51, 6.12487537e-51, 5.11059697e-51])),
 ('H2He', array([2.54717179e-11, 2.51775194e-11, 2.48865163e-11, ...,
         1.00000000e-33, 1.00000000e-33, 1.00000000e-33])),
 ('H2H', array([2.94469094e-13, 1.32506336e-17, 5.92089763e-22, ...,
         1.00000000e-33, 1.00000000e-33, 1.00000000e-33])),
 ('H2CH4', array([1.e-33, 1.e-33, 1.e-33, ..., 1.e-33, 1.e-33, 1.e-33])),
 ('H2N2', array([6.08533294e-33, 5.96022114e-33, 5.83759661e-33, ...,
         1.00000000e-33, 1.00000000e-33, 1.00000000e-33])),
 ('H2-', array([0., 0., 0., ..., 0., 0., 0.])),
 ('H-bf', array([0., 0., 0., ..., 0., 0., 0.])),
 ('H-ff', array([0., 0., 0., ..., 0., 0., 0.]))]

### Get Molecular Opacity

Molecular opacities are on a specific P-T grid so we book keep them by assigning indices to each pair e.g (1: 1e-6 bar, 75 K, 2:1e-6, 80K.. and so on) 

In [10]:
#get the PT grid with the corresponding grid 
cur.execute('SELECT ptid, pressure, temperature FROM molecular')
data= cur.fetchall()
pt_pairs = sorted(list(set(data)),key=lambda x: (x[0]) )
pt_pairs[0:10]#example of the first 10 PT pairs 

[(1, 0.0001, 100.0),
 (2, 0.0003, 100.0),
 (3, 0.001, 100.0),
 (4, 0.003, 100.0),
 (5, 0.01, 100.0),
 (6, 0.03, 100.0),
 (7, 0.1, 100.0),
 (8, 0.3, 100.0),
 (9, 1.0, 100.0),
 (10, 10.0, 100.0)]

In [11]:
#what molecules exist? 
cur.execute('SELECT molecule FROM molecular')
print(np.unique(cur.fetchall()))

['CH4' 'CO2' 'CrH' 'Cs' 'FeH' 'H2O' 'H2S' 'K' 'Li' 'NH3' 'Na' 'Rb' 'TiO'
 'VO']


In [12]:
# grab the opacity at a specific temp and pressure 
grab_p = 0.1 # bar 
grab_t = 100 # kelvin 
import math

#here's a little code to get out the correct pair (so we dont have to worry about getting the exact number right)
ind_pt = [min(pt_pairs, key=lambda c: math.hypot(c[1]- coordinate[0], c[2]-coordinate[1]))[0] 
          for coordinate in  zip([grab_p],[grab_t])]

cur.execute("""SELECT molecule,ptid,opacity 
            FROM molecular 
            WHERE molecule = ?
            AND ptid = ?""",('H2O',ind_pt[0]))
data= cur.fetchall()
data #gives you the molecule, ptid, and the opacity

[('H2O', 7, array([1.91787710e-26, 2.17703278e-27, 7.43505155e-28, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))]

In [13]:
grab_moles = ['H2O','CO2']
grab_p = [0.1,1,100] # bar 
grab_t = [100,200,700] # kelvin 

#here's a little code to get out the correct pair (so we dont have to worry about getting the exact number right)
ind_pt = [min(pt_pairs, key=lambda c: math.hypot(c[1]- coordinate[0], c[2]-coordinate[1]))[0] 
          for coordinate in  zip(grab_p,grab_t)]

cur.execute("""SELECT molecule,ptid,opacity 
            FROM molecular 
            WHERE molecule in {}
            AND ptid in {}""".format(str(tuple(grab_moles)), str(tuple(ind_pt))))
data= cur.fetchall()
data #gives you the molecule, ptid, and the opacity

[('CO2', 7, array([2.73962190e-33, 4.08753743e-34, 7.75603906e-34, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('CO2', 358, array([9.83365607e-31, 1.71966598e-30, 4.64583818e-31, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('CO2', 678, array([8.15678815e-29, 8.51294227e-29, 8.67542797e-29, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('H2O', 7, array([1.91787710e-26, 2.17703278e-27, 7.43505155e-28, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('H2O', 358, array([2.20469419e-25, 3.12302893e-26, 7.64981202e-26, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])),
 ('H2O', 678, array([2.42552885e-24, 2.57608350e-24, 2.52928641e-24, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))]

In [14]:
#Dont forget to close your connection!!!!
conn.close()

## Creating a New Database

**Note on molecule names**: Because ``picaso`` uses dict formatting to handle opacities, users can easily swap in different molecules. 

For example, if I wanted to include CO2-H2 CIA absorption, I can add ``CO2H2`` to the molecules list below. However, it is only quasi-automated in this regaurd. Please contact natasha.e.batalha@gmail.com if you are adding new CIA to the code. 

**Exceptions**: The exceptions to this are non-CIA continuum opacities. Right now, the other sources of continuum enabled are ``H2-``, ``H-bf`` and ``H-ff`` which have odd-ball formatting since they aren't simple two molecules. _Please let me know if you want to see another continuum source added_.

**Careful** with case sensitive molecules like **TiO**, **Na**. Make sure you get these right. 

In [15]:
db_filename = os.path.join('/Users/Natasha/Desktop/new_fake_opacity.db')
conn = sqlite3.connect(db_filename, detect_types=sqlite3.PARSE_DECLTYPES)
#same story with bytes and arrays
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

#tell sqlite what to do with an array
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

cur = conn.cursor()

### Build header, continuum, and molecular tables

It is **VERY** important that these tables are structured the same way. If you think something should be edited, ping natasha.e.batalha@gmail.com

In [16]:
#header
command="""DROP TABLE IF EXISTS header;
CREATE TABLE header (
    id INTEGER PRIMARY KEY,
    pressure_unit VARCHAR,
    temperature_unit VARCHAR,
    wavenumber_grid array,
    continuum_unit VARCHAR,
    molecular_unit VARCHAR
    );"""

cur.executescript(command)
conn.commit()
#molecular data table, note the existence of PTID which will be very important
command = """DROP TABLE IF EXISTS molecular;
CREATE TABLE molecular (
    id INTEGER PRIMARY KEY,
    ptid INTEGER,
    molecule VARCHAR ,
    pressure FLOAT,
    temperature FLOAT,
    opacity array);"""

cur.executescript(command)
conn.commit()
#continuum data table
command = """DROP TABLE IF EXISTS continuum;
CREATE TABLE continuum (
    id INTEGER PRIMARY KEY,
    molecule VARCHAR ,
    temperature FLOAT,
    opacity array);"""

cur.executescript(command)
conn.commit() #this commits the changes to the database

### Add header info (unit and wave grid info!)

The units **MUST** be the same. The wave grid can be whatever as long as it's consistent between continuum and molecular tables.

In [17]:
wave_grid = np.linspace(1e4/2, 1e4/0.5, 1000) #fake inverse cm wavenumber grid

cur.execute('INSERT INTO header (pressure_unit, temperature_unit, wavenumber_grid, continuum_unit,molecular_unit) values (?,?,?,?,?)', 
            ('bar','kelvin', np.array(wave_grid), 'cm-1 amagat-2', 'cm2/molecule'))
conn.commit()

### Insert continuum opacity to database

In [18]:
cia_temperature_grid = [100,300,500,700]
#insert continuum
for mol in ['H2H2', 'H2He', 'H2H', 'H2CH4', 'H2N2','H2-', 'H-bf', 'H-ff']:
    for T in cia_temperature_grid:
        OPACITY = wave_grid *0 + 1e-33 #INSERT YOUR OPACITY HERE
        cur.execute('INSERT INTO continuum (molecule, temperature, opacity) values (?,?,?)', (mol,float(T), OPACITY))
        conn.commit()

### Insert molecular opacity to database

Again, make sure that your molecules are **case-sensitive**: e.g. Sodium should be `Na` not `NA`

In [19]:
#create a fake PT grid 
pts=[]
for temp in [100,200,400]:
    for pres in [0.1, 1, 100]:
        pts += [[temp,pres]]
pts

[[100, 0.1],
 [100, 1],
 [100, 100],
 [200, 0.1],
 [200, 1],
 [200, 100],
 [400, 0.1],
 [400, 1],
 [400, 100]]

In [20]:
#insert molecular
for mol in ['H2O','CO2','CH4']:
    i = 1 #NOTE THIS INDEX HERE IS CRUCIAL! It will be how we quickly locate opacities 
    for T,P in pts:
        OPACITY = wave_grid *0 + 1e-33 #INSERT YOUR OPACITY HERE
        cur.execute('INSERT INTO molecular (ptid, molecule, temperature, pressure,opacity) values (?,?,?,?,?)', (i,mol,float(T),float(P), OPACITY))
        conn.commit()
        i+=1

In [21]:
#ALL DONE!!!
conn.close()